# Monetary and Fiscal Policy Coordination Notebook


In [1]:
import pandas as pd
import statsmodels as smf
import datetime as dt

from lets_plot import *
LetsPlot.setup_html()
from IPython.display import Image
import cairosvg

### Functions I made to make datetime conversion easier.

In [19]:
def strpquarter(i):
    i = str(i)
    i = i[0:4] + '-' + i[4:6]
    return pd.to_datetime(i)

In [28]:
def strptimer(i):
    i = str(i)
    i = i[0:4] + '-' + i[4:6]
    return pd.to_datetime(i)

## Taylor Rule Deviation:

$i_t = 2 + \pi_t + a(\pi_t-\pi^*)+b(y_t - y_t^*)$

+ From the formula above we need:

    + Interest Rate
    + Inflation Rate
    + Targetted Inflation Rate
    + Potential Output

### Interest Rates:

Sources: https://iftp.chinamoney.com.cn/english/bmklpr/, https://www.bis.org/statistics/cbpol/cbpol.xlsx

I need to quarterlyize the interest rates

In [126]:
# Static Variables

pstar = 3.0
ustar = 4.86

# Getting my dataset for interest rates

df = pd.DataFrame()
for i in range(0,8): # For between 2013-2020.
    t = pd.read_excel(f'../data/interest-rates/Loan_Prime_Rate_Historical_Data ({i}).xlsx',engine='openpyxl')
    t = t.loc[~((t['Date']=='Data source:')|(t['Date'].isnull())),:]
    df = pd.concat([df,t], ignore_index=True)

# Getting the dataset for benchmark rates 2010-2013.
t = pd.read_excel('../data/interest-rates/cbpol.xlsx', engine = 'openpyxl', sheet_name = 'Monthly Series')
t.columns = t.loc[2]
t = t.drop([0,1,2])
t = t.loc[(~(t['M:CN'].isnull())&(t['Period']<dt.datetime(2013,10,25))&(t['Period']>dt.datetime(2009,12,31))),['Period','M:CN']]
t['Date'] = t['Period']
t['1Y'] = t['M:CN']
df = pd.concat([df,t], ignore_index=True)

# Cleaning the data up.
df['Date'] = pd.to_datetime(df['Date'])
df['1Y'] = df['1Y'].astype(float)
df = df.rename(columns={'1Y':'1YearLPRRate'})
df = df.sort_values(by='Date',axis=0)
df = df.reset_index()
df = df.drop(columns =['Period', 'M:CN','5Y', 'index'])

# Changing to quarterly data
df['Quarter'] = pd.PeriodIndex(df['Date'],freq = 'Q')
df = df.groupby('Quarter', as_index=False).mean()
df['QuarterDate'] = df['Quarter'].apply(strpquarter)
df = df.set_index(df['QuarterDate'])
df = df.drop(columns = ['Date', 'QuarterDate'])

# Inflation Stat
df1 = pd.read_csv('../data/inflation-rates/CPALTT01CNQ659N.csv')
df1['observation_date'] = pd.to_datetime(df1['observation_date'])
df1 = df1.set_index(df1['observation_date'])
df1 = df1.drop(columns=['observation_date'])
df1 = df1['2010-01-01':'2020-12-31']
df['Inflation'] = df1['CPALTT01CNQ659N']

# GDP Stat
df1 = pd.read_csv('../data/real-GDP/CHNGDPNQDSMEI.csv')
df1['observation_date'] = pd.to_datetime(df1['observation_date'])
df1 = df1.set_index(df1['observation_date'])
df1 = df1.drop(columns=['observation_date'])
df1 = df1['2010-01-01':'2020-12-31']
df['NominalGDP'] = df1['CHNGDPNQDSMEI']

# Unemployment Stat
df1 = pd.read_excel('../data/unemployment-rates/imf-dm-export-modified-20250226.xls')
df1 = df1.transpose()
df1 = df1.reset_index()
df1.columns = df1.loc[0]
df1 = df1.rename(columns={'Unemployment Rate':'UnemploymentRate'})
df1 = df1.drop([0])
df1['Year'] = pd.to_datetime(df1['Year'], format='%Y')
df1 = df1.set_index(df1['Year'])
df1 = df1.drop(columns=['Year'])
df['UnemploymentRate'] = df1['UnemploymentRate']
df['UnemploymentRate'] = df['UnemploymentRate'].fillna(method='ffill')

# Ystar
df['Ystar'] = df['NominalGDP']/(1-0.4*(df['UnemploymentRate']-ustar))


df.to_csv('../data/chinese_coordinator.csv')
df

c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl'

,Quarter,1YearLPRRate,Inflation,NominalGDP,UnemploymentRate,Ystar
QuarterDate,,,,,,
2010-01-01,2010Q1,5.310000,2.071556,8.750130e+12,4.10,6.710222e+12
2010-04-01,2010Q2,5.310000,2.718078,9.934740e+12,4.10,7.618666e+12
2010-07-01,2010Q3,5.310000,3.264564,1.059637e+13,4.10,8.126051e+12
2010-10-01,2010Q4,5.643333,4.642333,1.193068e+13,4.10,9.149294e+12
2011-01-01,2011Q1,5.976667,5.223647,1.044699e+13,4.10,8.011495e+12
2011-04-01,2011Q2,6.310000,5.954095,1.188959e+13,4.10,9.117784e+12
2011-07-01,2011Q3,6.560000,6.438447,1.265622e+13,4.10,9.705690e+12
2011-10-01,2011Q4,6.560000,4.620924,1.380121e+13,4.10,1.058375e+13
2012-01-01,2012Q1,6.560000,3.727297,1.173576e+13,4.10,8.999816e+12


In [63]:
(
    ggplot(data = df.reset_index(), mapping = aes(x = 'Quarter Date'))
    + geom_line(aes(y = '1 Year LPR Rate'), manual_key = '1Y LPR')
    + geom_line(aes(y = 'Inflation'), color = 'red', manual_key = 'Inflation (%)')
)